In [1]:
import numpy as np
import cv2 as cv
import glob
# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
# Amount of corners in x and y direction (10, 7)
objp = np.zeros((7*10,3), np.float32)
objp[:,:2] = np.mgrid[0:10,0:7].T.reshape(-1,2)
# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.
images = glob.glob('calibration_320x240_v2/*.jpg')
print("Amount of images: ", len(images))
number_of_valid_images = 0
for fname in images:
    img = cv.imread(fname)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    # Find the chess board corners
    ret, corners = cv.findChessboardCorners(gray, (10,7), None)
    # If found, add object points, image points (after refining them)
    if ret == True:
        number_of_valid_images += 1
        objpoints.append(objp)
        corners2 = cv.cornerSubPix(gray,corners, (5,5), (-1,-1), criteria)
        imgpoints.append(corners2)
        # Draw and display the corners
        cv.drawChessboardCorners(img, (10,7), corners2, ret)
        cv.imshow('img', img)
        cv.waitKey(200)

print("Amount of valid images: ", number_of_valid_images)
cv.destroyAllWindows()

Amount of images:  40
Amount of valid images:  27


In [2]:
ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

In [9]:
# Undistort image
img = cv.imread('calibration_320x240_v2/14.jpg')
h,  w = img.shape[:2]
#newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))
# Undistort
dst = cv.undistort(img, mtx, dist, None, None)
# Crop the image
#x, y, w, h = roi
#dst = dst[y:y+h, x:x+w]
# Set old image next to it
img = np.hstack((dst, img))
cv.imshow('Undistorted vs Distorted', img)
cv.waitKey(0)
cv.destroyAllWindows()

In [3]:
print("Camera matrix: ", mtx)
print("Distortion coefficients: ", dist)

Camera matrix:  [[258.76440325   0.         159.73475449]
 [  0.         345.87451716 148.08435765]
 [  0.           0.           1.        ]]
Distortion coefficients:  [[-0.3832795   0.15241031 -0.0072451  -0.00629277 -0.11769157]]
